# **Incremental learning**

In [ ]:
import os

In [ ]:
# cloning the github repo with all the necessary code
if not os.path.isdir("./project_IL"):
  !git clone https://github.com/frattinfabio/project_IL.git

In [ ]:
from project_IL.model.IncrementalLearner import IncrementalLearner
from project_IL.params import get_params
from project_IL.data_handler.data_utils import load_data
from project_IL.classifiers.classifiers_utils import evaluate_incremental

## **Defining parameters**

In [ ]:
NUM_CLASSES = 100
NUM_GROUPS = 10
CLASSES_PER_GROUP = NUM_CLASSES // NUM_GROUPS
SPLITTING_SEED = 15
APPROACH = "COSINE" # possible values: ["FINETUNING", "LWF", "ICARL", "VARIATION", "COSINE"]

In [ ]:
# get the pre-defined params according to the decided approach
# modify those params for a different learning behaviour
# ex: modify approach_params["distillation_loss"] for a different distillation behaviour
train_params, approach_params = get_params(APPROACH) 

## **Main module**

In [ ]:
il = IncrementalLearner(NUM_CLASSES, NUM_GROUPS, SPLITTING_SEED, approach_params, train_params,)
classifier = approach_params["classifier"]

new_classes_accuracies = []
old_classes_accuracies = []
overall_accuracies = []

In [ ]:
for _ in range(il.num_groups):

  # going into the next incremental step
  il.step()
  # loading the new data
  train_dataloader, new_test_dataloader, old_test_dataloader = load_data(il.current_step, il.splitter.labels_split, il.train_params, il.exemplars)
  # updating the networks (main + old and ft for distillation if required)
  il.update_nets()
  # train the ft-net if using the variation approach
  if approach_params["use_variation"] and il.current_step > 0:
    train_dataloader_no_exemplars, _, _ = load_data(il.current_step, il.splitter.labels_split, il.train_params, None)
    il.train_ft(train_dataloader_no_exemplars)
  # train the main network
  il.train(train_dataloader)
  # updating the classifier with the new data and the new state of the network
  classifier.update(il.current_step, il.net, train_dataloader)
  # updating the exemplars set according to approach_params["exemplar_selection"]
  if approach_params["use_exemplars"]:
    il.update_exemplars()

  # evaluating the network both on old and new classes
  print("Classifying...")
  accuracies = evaluate_incremental(new_test_dataloader, old_test_dataloader, classifier)
  new_classes_accuracies.append(accuracies["new"])
  old_classes_accuracies.append(accuracies["old"])
  overall_accuracies.append(accuracies["overall"])
  print(f"Accuracies after having learned {il.n_known_classes} classes:")
  print(f"- current:\t{accuracies['new']:.3f}")
  if il.current_step > 0:
    print(f"- old:\t{accuracies['old']:.3f}")
    print(f"- overall:\t{accuracies['overall']:.3f}")
  print("")

## **Results visualization**




In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

results_df = pd.DataFrame(
    data = zip(range(CLASSES_PER_GROUP,NUM_CLASSES+CLASSES_PER_GROUP,CLASSES_PER_GROUP), new_classes_accuracies, old_classes_accuracies, overall_accuracies),
    columns = ["num_classes_learned", "new_classes_accuracy", "old_classes_accuracy", "overall_accuracy"])
results_df.set_index("num_classes_learned")

results_df.plot(x = "num_classes_learned", marker = 'o', figsize = (14,8))
plt.xlim(0,NUM_CLASSES+CLASSES_PER_GROUP)
plt.ylim(0,1)
plt.grid(True, axis = 'y')

In [ ]:
results_df.to_csv("./accuracy_results.csv", index = False)

In [ ]:
def get_predictions(classifier, test_dataloader): 
  predictions = []
  ground_truth = []
  for images, labels in test_dataloader:
    images = images.cuda()
    labels = labels.cuda()
    preds = classifier.classify(images)
    for label, pred in zip(labels.data, preds):
      ground_truth.append(label.item())
      predictions.append(pred.item())

  return ground_truth, predictions

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
from project_IL.data_handler.SubCIFAR import SubCIFAR
from torch.utils.data import DataLoader
import numpy as np

# plotting the predictions heatmap
full_test_dataset = SubCIFAR(labels_split = il.splitter.labels_split, labels = list(range(NUM_CLASSES)), train = False, transform = train_params["test_transform"])
full_test_dataloader =  DataLoader(full_test_dataset, batch_size = train_params["BATCH_SIZE"], num_workers = 4)
truth, pred = get_predictions(classifier, full_test_dataloader)
conf = confusion_matrix(truth, pred)

fig, ax = plt.subplots(figsize = (13,10))
sns.heatmap(np.log(conf+1), cmap="coolwarm", ax = ax)
plt.xlabel("Predicted class")
plt.ylabel("True Class")

In [ ]:
np.savetxt("preds.csv", pred, delimiter = ",")

In [ ]:
# histogram of number of predictions per class
fig, ax = plt.subplots(figsize = (16, 8))
labels, counts = np.unique(pred, return_counts=True)
plt.bar(labels, counts, align='center')
plt.xlabel("class")
plt.ylabel("num_predictions")
plt.hlines(y = 100, xmin = -0.5, xmax = 100-0.5, colors = 'r', label = "right number of predictions")
plt.show()

In [ ]:
import math
# histogram of number of predictions per group
group_pred = [math.floor(p/CLASSES_PER_GROUP) for p in pred]
fig, ax = plt.subplots(figsize = (16, 8))
labels, counts = np.unique(group_pred, return_counts=True)
plt.bar(labels, counts, align='center', width = 0.6)
plt.xlabel("group")
plt.ylabel("num_predictions")
plt.hlines(y = 100*CLASSES_PER_GROUP, xmin = -0.5, xmax = NUM_GROUPS-0.5, colors = 'r', label = "right number of predictions")
plt.gca().set_xticks(labels)
plt.show()

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(truth, pred, zero_division = 0, output_dict = True)
report_df = pd.DataFrame(report).transpose()

In [ ]:
report_df.to_csv("./classification_report.csv", index = False)